In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plot


In [ ]:
df= pd.read_csv("data/NYC_311_2k10_to_2k25.csv", nrows=5)
df.columns.to_list()

In [ ]:
cols_to_use = [
    "Unique Key", "Created Date", "Closed Date", "Agency", "Agency Name",
    "Complaint Type", "Descriptor", "Location Type", "Incident Zip",
    "City", "Borough", "Latitude", "Longitude", "Status", "Resolution Description"
]



df = pd.read_csv(
    "../data/NYC_311_2k10_to_2k25.csv",
    usecols=cols_to_use,
    low_memory=False
)

# import polars as pl

# df = pl.read_csv("../data/NYC_311_2k10_to_2k25.csv", columns=cols_to_use, try_parse_dates=True)



In [ ]:
df.columns

In [ ]:
df.info(memory_usage="deep")


In [ ]:
df["Created Date"] = pd.to_datetime(df["Created Date"], errors="coerce")
df["Closed Date"] = pd.to_datetime(df["Closed Date"], errors="coerce")


In [ ]:
df["resolution_time_hours"] = (
    (df["Closed Date"] - df["Created Date"]).dt.total_seconds() / 3600
)


In [ ]:
df = df[df["resolution_time_hours"].notna() & (df["resolution_time_hours"] >= 0)]


In [ ]:
df["Unique Key"] = pd.to_numeric(df["Unique Key"], downcast="integer")
df["Latitude"] = pd.to_numeric(df["Latitude"], downcast="float")
df["Longitude"] = pd.to_numeric(df["Longitude"], downcast="float")


In [ ]:
cat_cols = ["Agency", "Agency Name", "Complaint Type", "Descriptor",
            "Location Type", "Incident Zip", "City", "Status", "Borough"]

for col in cat_cols:
    df[col] = df[col].astype("category")


In [ ]:
df.info(memory_usage="deep")


In [ ]:
df.to_parquet("cleaned_nyc311_full.parquet")


In [ ]:
df

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.histplot(df["resolution_time_hours"], bins=100, kde=True)
plt.title("Distribution of Resolution Time (hours)")
plt.xlabel("Hours")
plt.ylabel("Frequency")
plt.xlim(0, df["resolution_time_hours"].quantile(0.95))  # Zoom in to remove long tail
plt.show()

print(df["resolution_time_hours"].describe())


In [ ]:
df['log_resolution_time'] = np.log1p(df['resolution_time_hours'])


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.boxplot(x=df['resolution_time_hours'])
plt.title("Boxplot of Resolution Time")
plt.show()


In [ ]:
threshold = df['resolution_time_hours'].quantile(0.99)
print(f"99th percentile: {threshold} hours")


In [ ]:
df_cleaned = df[df['resolution_time_hours'] <= threshold].copy()
print(f"Removed {(1 - len(df_cleaned)/len(df)) * 100:.2f}% of rows")


In [ ]:

plt.figure(figsize=(7, 5))
plt.hist(df_cleaned['resolution_time_hours'], bins=50, color='skyblue', edgecolor='black', alpha=0.7)
plt.title("Distribution of Resolution Time (hours)\nAfter Removing Top 1% Outliers")
plt.xlabel("Hours")
plt.ylabel("Frequency")
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import seaborn as sns

# Apply log1p to handle 0 values safely
df_cleaned['log_resolution_time'] = np.log1p(df_cleaned['resolution_time_hours'])

# Plot the transformed distribution
plt.figure(figsize=(7, 5))
sns.histplot(df_cleaned['log_resolution_time'], bins=50, kde=True, color='teal')
plt.title("Log-Scaled Distribution of Resolution Time (log1p)")
plt.xlabel("log(1 + Resolution Time in Hours)")
plt.ylabel("Frequency")
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()


In [ ]:
df.columns

In [ ]:
df_cleaned.describe()


In [ ]:
df_cleaned[df_cleaned["Borough"] == "Unspecified"].count()


In [ ]:
df_cleaned = df_cleaned[df_cleaned["Borough"] != "Unspecified"]


In [ ]:
df_cleaned.head(500)

In [ ]:
df_cleaned["Created Date"].min()


In [ ]:
df.columns

In [ ]:
df_cleaned.isnull().sum().sort_values(ascending=False)


In [ ]:
# Drop rows with missing lat/lon or borough
df_cleaned = df_cleaned.dropna(subset=["Latitude", "Longitude", "Borough"])

# Fill with 'Unknown' or 'Unspecified'
unknown_fill = ["Descriptor", "Resolution Description"]
unspecified_fill = ["Location Type", "City", "Incident Zip"]

for col in unknown_fill:
    if df_cleaned[col].dtype.name == "category":
        if "Unknown" not in df_cleaned[col].cat.categories:
            df_cleaned[col] = df_cleaned[col].cat.add_categories("Unknown")
    df_cleaned[col] = df_cleaned[col].fillna("Unknown")


for col in unspecified_fill:
    if df_cleaned[col].dtype.name == "category":
        if "Unspecified" not in df_cleaned[col].cat.categories:
            df_cleaned[col] = df_cleaned[col].cat.add_categories("Unspecified")
    df_cleaned[col] = df_cleaned[col].fillna("Unspecified")




In [ ]:
df_cleaned.isnull().sum().sort_values(ascending=False)


In [ ]:
# Ensure datetime format
df_cleaned["Created Date"] = pd.to_datetime(df_cleaned["Created Date"], errors="coerce")

# Extract features
df_cleaned["year"] = df_cleaned["Created Date"].dt.year
df_cleaned["month"] = df_cleaned["Created Date"].dt.month
df_cleaned["day_of_week"] = df_cleaned["Created Date"].dt.day_name()
df_cleaned["hour"] = df_cleaned["Created Date"].dt.hour
df_cleaned["is_weekend"] = df_cleaned["day_of_week"].isin(["Saturday", "Sunday"])


In [ ]:
df_cleaned[["Created Date", "year", "month", "day_of_week", "hour", "is_weekend"]].head()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

complaints_per_year = df_cleaned.groupby("year").size().reset_index(name="num_complaints")

sns.barplot(data=complaints_per_year, x="year", y="num_complaints")
plt.title("Total 311 Complaints per Year")
plt.ylabel("Complaint Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
avg_resolution_by_year = df_cleaned.groupby("year")["resolution_time_hours"].mean().reset_index()

sns.lineplot(data=avg_resolution_by_year, x="year", y="resolution_time_hours", marker="o")
plt.title("Average Resolution Time per Year")
plt.ylabel("Hours")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Complaints by weekday
sns.countplot(data=df_cleaned, x="day_of_week", order=[
    "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])
plt.title("Complaints by Day of Week")
plt.xticks(rotation=45)
plt.show()

# Complaints by hour
sns.histplot(data=df_cleaned, x="hour", bins=24, discrete=True)
plt.title("Complaints by Hour of Day")
plt.xlabel("Hour")
plt.show()


In [ ]:
complaints_by_borough_year = df_cleaned.groupby(["year", "Borough"]).size().reset_index(name="complaint_count")

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
sns.lineplot(data=complaints_by_borough_year, x="year", y="complaint_count", hue="Borough", marker="o")
plt.title("Total Complaints by Borough Over Years")
plt.ylabel("Complaint Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
resolution_by_borough_year = df_cleaned.groupby(["year", "Borough"])["resolution_time_hours"].mean().reset_index()

plt.figure(figsize=(12, 6))
sns.lineplot(data=resolution_by_borough_year, x="year", y="resolution_time_hours", hue="Borough", marker="o")
plt.title("Average Resolution Time by Borough Over Years")
plt.ylabel("Avg Resolution Time (Hours)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
df_cleaned["month"] = df_cleaned["Created Date"].dt.month
monthly_borough = df_cleaned.groupby(["year", "month", "Borough"]).size().reset_index(name="complaint_count")
monthly_borough["year_month"] = monthly_borough["year"].astype(str) + "-" + monthly_borough["month"].astype(str).str.zfill(2)

plt.figure(figsize=(14, 6))
sns.lineplot(data=monthly_borough, x="year_month", y="complaint_count", hue="Borough")
plt.title("Monthly Complaint Volume per Borough")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd

# Approximate NYC borough populations (source: Census + interpolated estimates)
data = {
    "Borough": ["BRONX", "BROOKLYN", "MANHATTAN", "QUEENS", "STATEN ISLAND"] * 5,
    "year": [2021]*5 + [2022]*5 + [2023]*5 + [2024]*5 + [2025]*5,
    "Population": [
        1455720, 2582830, 1638281, 2321580, 495747,  # 2021
        1458000, 2590000, 1640000, 2330000, 497000,  # 2022
        1460000, 2600000, 1642000, 2340000, 498500,  # 2023
        1462000, 2610000, 1644000, 2350000, 500000,  # 2024
        1464000, 2620000, 1646000, 2360000, 501500   # 2025
    ]
}

population_df = pd.DataFrame(data)


In [ ]:
# Group complaints by borough and year
complaints_by_borough_year = df_cleaned.groupby(["year", "Borough"]).size().reset_index(name="complaint_count")

# Merge with population data
merged = complaints_by_borough_year.merge(population_df, on=["year", "Borough"], how="left")

# Calculate normalized metric
merged["complaints_per_1000"] = (merged["complaint_count"] / merged["Population"]) * 1000


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
sns.lineplot(data=merged, x="year", y="complaints_per_1000", hue="Borough", marker="o")
plt.title("Normalized Complaint Volume (per 1,000 People)")
plt.ylabel("Complaints per 1,000 Residents")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
df_cleaned.columns

In [ ]:
top_complaints = df_cleaned["Complaint Type"].value_counts().head(10).index
df_top = df_cleaned[df_cleaned["Complaint Type"].isin(top_complaints)]


In [ ]:
grouped_resolution = (
    df_top.groupby(["Complaint Type", "Borough"])["resolution_time_hours"]
    .mean()
    .reset_index()
)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

pivot = grouped_resolution.pivot(index="Complaint Type", columns="Borough", values="resolution_time_hours")

plt.figure(figsize=(12, 6))
sns.heatmap(pivot, annot=True, fmt=".1f", cmap="YlOrRd")
plt.title("Avg Resolution Time (hrs) by Complaint Type and Borough")
plt.ylabel("Complaint Type")
plt.xlabel("Borough")
plt.tight_layout()
plt.show()


In [ ]:
# Use the same grouped_resolution DataFrame from the heatmap
disparity_df = (
    grouped_resolution.groupby("Complaint Type")["resolution_time_hours"]
    .agg(["min", "max", "std"])
    .reset_index()
)

# Add range column
disparity_df["range"] = disparity_df["max"] - disparity_df["min"]

# Sort by range (largest disparity first)
top_disparity = disparity_df.sort_values(by="range", ascending=False).head(10)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
sns.barplot(data=top_disparity, x="range", y="Complaint Type", palette="coolwarm")
plt.title("Top 10 Complaint Types with Highest Borough Disparity (Resolution Time Range)")
plt.xlabel("Resolution Time Range (Hours)")
plt.ylabel("Complaint Type")
plt.tight_layout()
plt.show()


In [ ]:
# After computing `disparity_df`:
top_disparity = disparity_df.sort_values(by="range", ascending=False).head(10)




In [ ]:
# Recompute top disparity complaint types
top_disparity = (
    disparity_df.sort_values(by="range", ascending=False)
    .head(10)
    .copy()
)

# Confirm that only 10 complaint types exist
print(top_disparity[["Complaint Type", "range"]])

# Now plot ONLY the top 10
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.barplot(
    data=top_disparity,
    x="range",
    y="Complaint Type",
    palette="coolwarm"
)
plt.title("Top 10 Complaint Types with Highest Borough Disparity\n(Resolution Time Range)")
plt.xlabel("Resolution Time Range (Hours)")
plt.ylabel("Complaint Type")
plt.tight_layout()
plt.show()


In [ ]:
focus_type = "UNSANITARY CONDITION"  # or "Water System"

subset = grouped_resolution[grouped_resolution["Complaint Type"] == focus_type]

import seaborn as sns
sns.barplot(data=subset, x="resolution_time_hours", y="Borough", palette="mako")
plt.title(f"Avg Resolution Time by Borough — {focus_type}")
plt.xlabel("Resolution Time (hrs)")
plt.tight_layout()
plt.show()


In [ ]:
high_disparity_types = top_disparity["Complaint Type"].tolist()

df_cleaned["is_high_disparity_type"] = df_cleaned["Complaint Type"].isin(high_disparity_types)


In [ ]:
# Step 1: Avg per type (global)
global_avg = df_cleaned.groupby("Complaint Type")["resolution_time_hours"].mean().reset_index()
global_avg.rename(columns={"resolution_time_hours": "global_avg"}, inplace=True)

# Step 2: Avg per borough + type
borough_avg = df_cleaned.groupby(["Complaint Type", "Borough"])["resolution_time_hours"].mean().reset_index()
borough_avg = borough_avg.merge(global_avg, on="Complaint Type", how="left")
borough_avg["delay_bias"] = borough_avg["resolution_time_hours"] - borough_avg["global_avg"]


In [ ]:
from scipy.stats import f_oneway

# Prepare groups
groups = [df_cleaned[df_cleaned["Borough"] == b]["resolution_time_hours"].dropna().values
          for b in ["BRONX", "BROOKLYN", "MANHATTAN", "QUEENS", "STATEN ISLAND"]]

# Run ANOVA
f_stat, p_value = f_oneway(*groups)
print(f"ANOVA Results → F-statistic: {f_stat:.2f}, p-value: {p_value:.5f}")



In [ ]:
df_cleaned["Borough"].value_counts()


In [ ]:
df_cleaned.groupby("Borough")["resolution_time_hours"].count()


In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

tukey = pairwise_tukeyhsd(
    endog=df_cleaned["resolution_time_hours"],
    groups=df_cleaned["Borough"],
    alpha=0.05
)

print(tukey.summary())


In [ ]:
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.stats.multicomp import MultiComparison

mc = MultiComparison(df_cleaned["resolution_time_hours"], df_cleaned["Borough"])
result = mc.tukeyhsd()

fig = result.plot_simultaneous(figsize=(10, 5))
plt.title("Tukey HSD: Mean Resolution Time Differences Across Boroughs")
plt.grid(True)
plt.show()
